In [ ]:
import torch
from torchvision.models.segmentation import LRASPP_MobileNet_V3_Large_Weights

torch.random.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [ ]:
from collections import deque


class FixedSizeQueue:
    def __init__(self, max_length):
        self.queue = deque([0] * max_length, maxlen=max_length)

    def enqueue(self, item):
        self.queue.append(item)

    def get_average(self):
        return sum(self.queue) / len(self.queue)

    def get_last(self):
        return self.queue[-1]

    def __str__(self):
        return str(list(self.queue))

In [ ]:
class Creature:
    def __init__(self, tps):
        self.ticks_per_second = tps
        self.time_resolution = 1 / self.ticks_per_second
        self.mass = 0.1
        self.gravity = 9.81
        self.friction_coefficient = 0.1
        self.f_friction = self.friction_coefficient * self.mass * self.gravity
        self.battery = 100
        self.velocity_history = FixedSizeQueue(self.ticks_per_second * 1)

    def get_data(self):
        return self.battery, self.velocity_history.get_last()

    def battery_discharge(self, x: float):
        """Given a certain requested force, by how much to lower the battery"""
        return ((x / 5) ** 2) * self.time_resolution

    def apply_force(self, force: float) -> None:
        self.battery = min(self.battery + (1 * self.time_resolution), 100)
        battery_to_discharge = self.battery_discharge(abs(force))
        if battery_to_discharge <= self.battery:
            v_old = self.velocity_history.get_last()
            friction = -self.f_friction if (v_old > 0) else (self.f_friction if v_old < 0 else 0)
            f_net = force + friction
            """ friction acts opposite current velocity """
            friction_deceleration = True if abs(force) < self.f_friction else False
            """ is the object decelerating due to friction and not applied force? if so then it naturally slows to a halt as the force is weaker then friction, it will not move the other way """
            delta_v = (f_net * self.time_resolution) / self.mass
            v_new = v_old + delta_v
            v_new = 0 if (friction_deceleration and v_new * v_old < 0) else v_new
            """ checks if v_old to v_new has made a switch from pos to neg or vice versa; if decelerating due to friction, then v_new must be 0 as stated above """
            self.velocity_history.enqueue(v_new)
            self.battery -= battery_to_discharge * self.time_resolution
        else:
            self.velocity_history.enqueue(0)

In [ ]:
import torch
from torch import nn


class Model(nn.Module):
    def __init__(self, latent_size: int, num_inputs: int) -> None:
        super().__init__()

        self.prev_latent = torch.zeros(latent_size)
        self.prev_force = torch.zeros(1)

        self.layer_norm = nn.LayerNorm(latent_size)

        self.mean_history = []
        self.std_history = []

        self.encoder = nn.Sequential(
            nn.Linear(latent_size + num_inputs, latent_size * 2),
            nn.LeakyReLU(),
            nn.Linear(latent_size * 2, latent_size),
        )

        self.mean = nn.Sequential(
            nn.Linear(latent_size, latent_size * 2),
            nn.LeakyReLU(),
            nn.Linear(latent_size * 2, 1)
        )

        self.std = nn.Sequential(
            nn.Linear(latent_size, latent_size * 2),
            nn.LeakyReLU(),
            nn.Linear(latent_size * 2, 1),
            nn.Softplus()
        )

        self.predictor = nn.Sequential(
            nn.Linear(latent_size + 1, latent_size * 2),
            nn.LeakyReLU(),
            nn.Linear(latent_size * 2, latent_size),
        )

    def update_hidden_states(self, force, latent):
        self.prev_force = force
        self.prev_latent = latent

    def forward(self, inputs):
        self.prev_force = self.prev_force.detach()
        self.prev_latent = self.prev_latent.detach()

        tmp_1 = torch.cat([inputs, self.prev_latent], dim=0)
        curr_latent = self.prev_latent + self.encoder(tmp_1)
        curr_latent = self.layer_norm(curr_latent)

        y1_mean = self.mean(curr_latent)
        y1_std = self.std(curr_latent) + 0.001
        self.mean_history.append(y1_mean.item())
        self.std_history.append(y1_std.item())

        tmp_2 = torch.cat([self.prev_latent, self.prev_force], dim=0)
        predicted_latent = self.prev_latent + self.predictor(tmp_2)
        predicted_latent = self.layer_norm(predicted_latent)

        dist = torch.distributions.Normal(y1_mean, y1_std)
        y1 = dist.rsample()
        z = torch.abs(y1 - y1_mean) / y1_std
        standard_normal = torch.distributions.Normal(torch.tensor(0.0), torch.tensor(1.0))
        probability = (1 - standard_normal.cdf(z)) * 2

        return curr_latent, predicted_latent, y1, y1_std, probability

In [ ]:
import torch
from torch import nn


class RewardFunction(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self, real_latent, predicted_latent, velocity_reward, probability):
        euclid_dist = torch.cdist(real_latent, predicted_latent)
        surprise_factor = -euclid_dist
        weighted_velocity_reward = velocity_reward
        composite_reward = weighted_velocity_reward + surprise_factor
        loss = torch.exp(-composite_reward)
        return loss, weighted_velocity_reward, surprise_factor, euclid_dist

In [ ]:
SECONDS = 100
LR_PER_SECOND = 1
TICKS_PER_SECOND = 100

robot = Creature(
    tps=TICKS_PER_SECOND
)
model = Model(
    latent_size=64,
    num_inputs=2,
)

LEARNING_RATE = LR_PER_SECOND * (1 / TICKS_PER_SECOND)
TIMESTEPS = SECONDS * TICKS_PER_SECOND

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
reward_fn = RewardFunction()

velocity_hist = []
avg_velocity_hist = []
battery_hist = []
loss_history = []
surprise_factor_history = []
v_reward_history = []
s_reward_history = []
total_reward_history = []

model.train()

for timestep in range(TIMESTEPS):
    optimizer.zero_grad()

    battery, v_last = robot.get_data()

    data = torch.Tensor([battery / 100, v_last])

    curr_lat, pred_lat, force, std, prob = model(data)

    robot.apply_force(force.item())
    v_curr = robot.velocity_history.get_last()
    v_avg = robot.velocity_history.get_average()

    velocity_hist.append(v_curr)
    avg_velocity_hist.append(v_avg)
    battery_hist.append(robot.battery)

    loss, v_reward, s_reward, s_factor = reward_fn(
        real_latent=curr_lat.unsqueeze(0),
        predicted_latent=pred_lat.unsqueeze(0),
        velocity_reward=torch.Tensor([v_curr]),
        probability=prob
    )
    loss.backward()

    loss_history.append(loss.item())
    surprise_factor_history.append(s_factor.item())
    v_reward_history.append(v_reward.item())
    s_reward_history.append(s_reward.item())
    total_reward_history.append(v_reward.item() + s_reward.item())

    optimizer.step()

    model.update_hidden_states(
        force=force,
        latent=curr_lat,
    )

    if (timestep + 1) % 100 == 0:
        print(f"{(timestep + 1) / TICKS_PER_SECOND:.3f}s - {((timestep + 1) / TIMESTEPS) * 100:.2f}%", end=" ")
        print(f"| V {v_curr:.5f}", end=" ")
        print(f"| AV {v_avg:.5f}", end=" ")
        print(f"| SF {s_factor.item():.5f}", end=" ")
        print(f"| VR {v_reward.item():.3f}", end=" ")
        print(f"| SR {s_reward.item():.3f}", end=" ")
        print(f"| Pr {prob.item():.3f}", end=" ")
        print(f"| L {loss.item():.8f}", end="\n")

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(4, 2, figsize=(10, 16))

axes = axes.flatten()

axes[0].plot(velocity_hist)
axes[0].set_title('Velocity')

axes[1].plot(avg_velocity_hist)
axes[1].set_title('Average Velocity')

axes[2].plot(model.mean_history)
axes[2].set_title('Mean')

axes[3].plot(model.std_history)
axes[3].set_title('Standard deviation')

axes[4].plot(loss_history)
axes[4].set_title('Loss (lower is better)')

axes[5].plot(surprise_factor_history)
axes[5].set_title('Surprise Factor')

axes[6].plot(battery_hist)
axes[6].set_title('Battery')

axes[7].plot(v_reward_history, label="Velocity")
axes[7].plot(s_reward_history, label="S factor")
axes[7].plot(total_reward_history, label="Total")
axes[7].set_title('Reward')
axes[7].legend()

plt.tight_layout()

plt.show()